In [1]:
#first import libraries
import pandas as pd
import numpy as np

#read in the csv kept at the same link each day
filename = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
cases = pd.read_csv(filename)

#look at first five rows
cases.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [2]:
#get week ending date
from datetime import datetime, timedelta
from pandas.tseries.offsets import *

#reference: https://stackoverflow.com/questions/41767102/pandas-convert-week-date-to-weekend-friday-date
#calculate new columns for week ending

cases['DATE_obj'] = pd.to_datetime(cases['date'])
cases['week_ending'] = cases['DATE_obj'].where(cases['DATE_obj'] == (( cases['DATE_obj'] + Week(weekday=5) ) - Week()), cases['DATE_obj'] + Week(weekday=5))

#convert week_end column to string for compliance in sql database
cases['week_ending'] = cases['week_ending'].dt.strftime('%Y-%m-%d')

#check results
cases.head(50)

,date,state,fips,cases,deaths,DATE_obj,week_ending
0,2020-01-21,Washington,53,1,0,2020-01-21,2020-01-25
1,2020-01-22,Washington,53,1,0,2020-01-22,2020-01-25
2,2020-01-23,Washington,53,1,0,2020-01-23,2020-01-25
3,2020-01-24,Illinois,17,1,0,2020-01-24,2020-01-25
4,2020-01-24,Washington,53,1,0,2020-01-24,2020-01-25
5,2020-01-25,California,6,1,0,2020-01-25,2020-01-25
6,2020-01-25,Illinois,17,1,0,2020-01-25,2020-01-25
7,2020-01-25,Washington,53,1,0,2020-01-25,2020-01-25
8,2020-01-26,Arizona,4,1,0,2020-01-26,2020-02-01
9,2020-01-26,California,6,2,0,2020-01-26,2020-02-01


In [3]:
#now send this data to SQL!
#reference: https://stackoverflow.com/questions/30631325/writing-to-mysql-database-with-pandas-using-sqlalchemy-to-sql
import pymysql
from sqlalchemy import create_engine

user = 'admin'
passw = 'dude1990'
host =  'group3final.cx1foeolbgvg.us-east-2.rds.amazonaws.com'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
port = 3306 
database = 'stock_data'

mydb = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

cases.to_sql(name='corona_cases', con=mydb, if_exists = 'replace', index=False)

,date,state,fips,cases,deaths,DATE_obj,week_ending
0,2020-01-21,Washington,53,1,0,2020-01-21,2020-01-25
1,2020-01-22,Washington,53,1,0,2020-01-22,2020-01-25
2,2020-01-23,Washington,53,1,0,2020-01-23,2020-01-25
3,2020-01-24,Illinois,17,1,0,2020-01-24,2020-01-25
4,2020-01-24,Washington,53,1,0,2020-01-24,2020-01-25
...,...,...,...,...,...,...,...
4244,2020-05-18,Virginia,51,31140,1014,2020-05-18,2020-05-23
4245,2020-05-18,Washington,53,19838,1020,2020-05-18,2020-05-23
4246,2020-05-18,West Virginia,54,1502,68,2020-05-18,2020-05-23
4247,2020-05-18,Wisconsin,55,12722,459,2020-05-18,2020-05-23
